## Import stuff

In [1]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# import torch 
# import torch.jit
# import torch.nn as nn 
# import torchvision
# from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())




def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.8.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9076695435710838380
xla_global_id: -1
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Load training data from csv


In [2]:
df = pd.read_csv("train_features.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(df.iloc[:, :].values,  dtype=float)
Y = np.array(labels_df.iloc[:, :].values,  dtype=float)

#normalize
# Y[:,13] = (Y[:,13]-27)/73

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)



X shape:  (59515, 2560)
Y shape:  (59515, 1)


## The function that trains and rates models

In [3]:
def find_best_model(X_train, Y_train, X_valid, Y_valid, X_test):
    models = []

 
    # print("training 64+64 relu")
    # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    # model = tf.keras.Sequential([
    #     layers.Dense(64, activation=tf.keras.activations.relu),
    #     layers.Dense(64, activation=tf.keras.activations.relu),
    #     layers.Dense(1, activation='sigmoid')
    # ])
    # model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01))
    # history = model.fit(X_train, Y_train, epochs=50, verbose=False,callbacks=[callback])
    # print(len(history.history['loss']))
    # models.append(model)


    print("training ridge")
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, Y_train)
    models.append(ridge)
    

    
    print("done training all models")

    best_score = 0
    best_model = None
    probabilistic_models = ["MLPClassifier", "SVC", "RandomForestClassifier", "HistGradientBoostingClassifier", "GradientBoostingClassifier", "ExtraTreesClassifier", "RandomForestRegressor", "DecisionTreeClassifier", "GaussianNB", "AdaBoostClassifier", "QuadraticDiscriminantAnalysis", "GaussianProcessClassifier"]
    for model in models:
        model_type = str(type(model)).split(".")[-1].split("'")[0]
        # print("testing model ", model_type)
        #if the model is probabilistic
        if model_type in probabilistic_models:
            pred = model.predict_proba(X_valid)[:,1]
        else:
            pred = model.predict(X_valid)

        np.clip(pred, 0.0, 1.0, out=pred)

        score = metrics.roc_auc_score(Y_valid, pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model))

    #doesn't seem to work
    # if str(type(best_model)).count("Sequential") > 0:
    #     best_model.fit(X,Y[:,col], epochs=15, verbose=False)
    # else:
    #     best_model.fit(X, Y[:,col])

    predictions = best_model.predict(X_test)
    
    # print("predictions shape: ", predictions.shape)

    return predictions

## Writing results

In [6]:
# X_test = pd.read_csv("test_features.csv")
# X_test = pd.read_csv("train_features.csv")
X_test = []
# X_test = np.array(X_test.iloc[:, :].values,  dtype=float)


# pred = find_best_model(X, Y, X, Y, X_test)
pred = find_best_model(X[:int(X.shape[0]*0.8)], Y[:int(Y.shape[0]*0.8)], X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test)

print("pred shape: ", pred.shape)
pred = pred.reshape(pred.shape[0],1)


training ridge
done training all models
score for <class 'sklearn.linear_model._ridge.Ridge'> :  0.610316412416925
best model:  <class 'sklearn.linear_model._ridge.Ridge'>


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:

# pred = np.array(np.round(pred, 0), dtype=int)
# save predictions to txt file
np.savetxt("submission.txt", pred, delimiter="\n")


